In [ ]:
# | default_exp _code_generator.app_and_test_generator

In [ ]:
# | export

from typing import *
import time
import importlib.util
from tempfile import TemporaryDirectory
from pathlib import Path
import platform
from collections import defaultdict
import subprocess  # nosec: B404: Consider possible security implications associated with the subprocess module.

from yaspin import yaspin

from faststream_gen._components.logger import get_logger
from faststream_gen._code_generator.chat import CustomAIChat, ValidateAndFixResponse
from faststream_gen._code_generator.helper import (
    write_file_contents,
    read_file_contents,
    validate_python_code,
    retry_on_error,
    set_cwd
)
from faststream_gen._code_generator.prompts import APP_AND_TEST_GENERATION_PROMPT
from faststream_gen._code_generator.constants import (
    APPLICATION_FILE_PATH,
    TEST_FILE_PATH,
    STEP_LOG_DIR_NAMES,
    LOGS_DIR_NAME,
)

from faststream_gen._code_generator.constants import OpenAIModel

In [ ]:
from contextlib import contextmanager
import unittest.mock

import pytest
import openai

from faststream_gen._components.logger import suppress_timestamps
from faststream_gen._code_generator.helper import mock_openai_create

In [ ]:
# | export

logger = get_logger(__name__)

In [ ]:
suppress_timestamps()
logger = get_logger(__name__, level=30)
logger.info("ok")

In [ ]:
# | export

_code_fix_prompt = """
Your task is to correct the provided code. Your response should consist solely of valid Python code. You must follow the below rules while responding:

- Do not include explanations or wrap your response in ```python tags.
"""

def _fix_generated_code(s: str) -> str:
    ai = CustomAIChat(
        params={
            "temperature": 0.2,
        },
        model=OpenAIModel.gpt3.value,
        user_prompt=_code_fix_prompt,
    )
    response, usage = ai(s) # todo: add this usage to total usage
    return response

In [ ]:
# | notest

fixture = """
print("hi)
"""

expected = """print("hi")"""

actual = _fix_generated_code(fixture)
print(actual)
assert actual == expected

print("hi")


In [ ]:
# | export


def _split_app_and_test_code(response: str) -> Tuple[str, str]:
    app_code, test_code = response.split("### application.py ###")[1].split(
        "### test.py ###"
    )
    return app_code, test_code


def _validate_response(
    response: str, output_directory: str, **kwargs: Dict[str, Any]
) -> Tuple[List[str], str]:
    try:
        app_code, test_code = _split_app_and_test_code(response)
    except (IndexError, ValueError) as e:
        return (
            ["Please add ### application.py ### and ### test.py ### in your response"],
            response,
        )

    app_code = app_code.replace("### application.py ###", "").strip()
    test_code = test_code.strip()

    fixed_app_code = _fix_generated_code(app_code)
    fixed_test_code = test_code.replace("from application import ", "from app.application import ") #_fix_generated_code(test_code)

    app_file_name = Path(output_directory) / APPLICATION_FILE_PATH
    test_file_name = Path(output_directory) / TEST_FILE_PATH

    write_file_contents(str(app_file_name), fixed_app_code)
    write_file_contents(str(test_file_name), fixed_test_code)

    with set_cwd(output_directory):
        cmd = ["pytest", "--tb=short"]
        # nosemgrep: python.lang.security.audit.subprocess-shell-true.subprocess-shell-true
        p = subprocess.run(  # nosec: B602, B603 subprocess call - check for execution of untrusted input.
            cmd,
            stderr=subprocess.PIPE,
            stdout=subprocess.PIPE,
            shell=True if platform.system() == "Windows" else False,
        )
    if p.returncode != 0:
        response = f"### application.py ###\n{fixed_app_code}\n\n### test.py ###\n{fixed_test_code}\n"
        return ([str(p.stdout.decode("utf-8"))], response)

    return ([], "")

In [ ]:
fixture_response = """
### application.py ###

print('hi')

### test.py ###
def test_always_passes():
    assert True
"""
test_response = """
def test_always_passes():
    assert True
"""
with TemporaryDirectory() as d:
    expected = ([], "")
    with mock_openai_create(test_response):
        actual = _validate_response(fixture_response, d)
        print(actual)
        assert actual == expected

([], '')


In [ ]:
fixture_response = """
### application.py ##

print('hi')

### test.py ###
def test_always_passes():
    assert True
"""
test_response = """
def test_always_passes():
    assert True
"""
with TemporaryDirectory() as d:
    expected = (['Please add ### application.py ### and ### test.py ### in your response'], "\n### application.py ##\n\nprint('hi')\n\n### test.py ###\ndef test_always_passes():\n    assert True\n")
    with mock_openai_create(test_response):
        actual = _validate_response(fixture_response, d)
        print(actual)
        assert actual == expected, actual

(['Please add ### application.py ### and ### test.py ### in your response'], "\n### application.py ##\n\nprint('hi')\n\n### test.py ###\ndef test_always_passes():\n    assert True\n")


In [ ]:
fixture_response = """
### application.py ###

print('hi')

### test.py ###

def test_always_fails():
    assert False
"""
fixture_app_code = "print('hi')"

test_response = """
def test_always_fails():
    assert False
"""

with TemporaryDirectory() as d:
    with mock_openai_create(test_response):
        actual = _validate_response(fixture_response, d)
        print(actual[0])
        assert actual != [], actual
        print("OK")

['\x1b============================= test session starts ==============================\x1b\nplatform linux -- Python 3.11.5, pytest-7.4.2, pluggy-1.3.0\nrootdir: /tmp/tmpjb8y5nsv\nplugins: anyio-3.7.1, asyncio-0.21.1\nasyncio: mode=Mode.STRICT\ncollected 1 item\n\ntests/test_application.py \x1bF\x1b\x1b                                              [100%]\x1b\n\n=================================== FAILURES ===================================\n\x1b\x1b______________________________ test_always_fails _______________________________\x1b\n\x1b\x1btests/test_application.py\x1b:2: in test_always_fails\n    \x1bassert\x1b \x1bFalse\x1b\x1b\x1b\n\x1b\x1bE   assert False\x1b\n\x1b\x1b=========================== short test summary info ============================\x1b\n\x1bFAILED\x1b tests/test_application.py::\x1btest_always_fails\x1b - assert False\n\x1b============================== \x1b\x1b1 failed\x1b\x1b in 0.04s\x1b\x1b ===============================\x1b\n']
OK


In [ ]:
# | export


@retry_on_error()  # type: ignore
def _generate(
    model: str,
    prompt: str,
    app_skeleton: str,
    total_usage: List[Dict[str, int]],
    output_directory: str,
    **kwargs,
) -> Tuple[str, List[Dict[str, int]], bool]:
    test_generator = CustomAIChat(
        params={
            "temperature": 0.2,
        },
        model=model,
        user_prompt=prompt,
    )
    test_validator = ValidateAndFixResponse(test_generator, _validate_response)
    validator_result = test_validator.fix(
        app_skeleton,
        total_usage,
        STEP_LOG_DIR_NAMES["app"],
        str(output_directory),
        **kwargs,
    )
    
    return (
        (validator_result, True) # type: ignore
        if isinstance(validator_result[-1], defaultdict)
        else validator_result
    )

In [ ]:
model = OpenAIModel.gpt3.value
prompt = "Some valid prompt"
app_skeleton = "some app skeleton"
total_usage = []

test_response = """
### application.py ###

print('some valid python code')
    
### test.py ###

from app import application

def test_always_passes():
    assert True

"""


with TemporaryDirectory() as d:
    test_file_path = Path(d) / TEST_FILE_PATH
    test_file_path.parent.mkdir(parents=True, exist_ok=True)
    init_file_path = test_file_path.parent / "__init__.py"
    init_file_path.touch()
    with mock_openai_create(test_response):
        total_usage, is_valid_app_code = _generate(
            model, prompt, app_skeleton, total_usage, d
        )
        
    print(is_valid_app_code)
    
    assert is_valid_app_code
    assert isinstance(is_valid_app_code, bool)

True


In [ ]:
model = OpenAIModel.gpt3.value
prompt = "Some invalid prompt"
app_skeleton = "some invalid app skeleton"
total_usage = []

test_response = """
### application.py ###

print("invalid app code")
    
### test.py ###

import pytest

from faststream.kafka import TestKafkaBroker

from .app import CourseUpdates, broker, on_course_update


print("invalid test code")

"""

with TemporaryDirectory() as d:
    test_file_path = Path(d) / TEST_FILE_PATH
    test_file_path.parent.mkdir(parents=True, exist_ok=True)
    init_file_path = test_file_path.parent / "__init__.py"
    init_file_path.touch()
    with mock_openai_create(test_response):
        total_usage, is_valid_app_code = _generate(
            model, prompt, app_skeleton, total_usage, d
        )
        print(is_valid_app_code)
        assert not is_valid_app_code
        assert isinstance(is_valid_app_code, bool)

False


In [ ]:
# | export


def generate_app_and_test(
    description: str,
    model: str,
    output_directory: str,
    total_usage: List[Dict[str, int]],
    relevant_prompt_examples: str,
) -> Tuple[List[Dict[str, int]], bool]:
    """Generate integration test for the FastStream app

    Args:
        description: Validated User application description
        code_gen_directory: The directory containing the generated files.
        relevant_prompt_examples: Relevant examples to add in the prompts.

    Returns:
        The generated integration test code for the application
    """
    logger.info("==== Skeleton to App and Test Generation ====")
    with yaspin(
        text="Generating application and tests (usually takes around 30 to 90 seconds)...",
        color="cyan",
        spinner="clock",
    ) as sp:
        app_skeleton_file_name = Path(output_directory) / APPLICATION_FILE_PATH
        app_skeleton = read_file_contents(str(app_skeleton_file_name))

        prompt = (
            APP_AND_TEST_GENERATION_PROMPT.replace(
                "==== REPLACE WITH APP DESCRIPTION ====", description
            )
            .replace("==== RELEVANT EXAMPLES GOES HERE ====", relevant_prompt_examples)
            .replace("from .app import", "from app.application import")
        )

        total_usage, is_valid_app_code = _generate(
            model, prompt, app_skeleton, total_usage, output_directory
        )
        
        sp.text = ""
        if is_valid_app_code:
            message = " ✔ The application and the test files are generated."
        else:
            message = " ✘ Error: Failed to generate a valid application and test code."
            sp.color = "red"

        sp.ok(message)

        return total_usage, is_valid_app_code

In [ ]:
fixture_skeleton_code = """
from pydantic import BaseModel, Field

from faststream import FastStream, Logger
from faststream.kafka import KafkaBroker


class Product(BaseModel):
    product_name: str = Field(..., description="Name of the product")
    currency: str = Field(..., description="Currency of the price")
    price: float = Field(..., description="Price of the product")


broker = KafkaBroker("localhost:9092")
app = FastStream(broker)


@broker.publisher("change_currency")
@broker.subscriber("store_product")
async def on_store_product(product: Product, logger: Logger) -> Product:
    '''Processes a message from 'store_product' topic, changes currency to 'EUR' and divides price by 7.5 if currency is 'HRK'.

    Instructions:
    1. Consume a message from 'store_product' topic.
    2. Log the consumed message using logger.info.
    3. Check if the currency attribute is set to 'HRK'.
    4. If the currency is 'HRK', change the currency to 'EUR' and divide the price by 7.5.
    5. If the currency is not 'HRK', do not modify the original message.
    6. Publish the consumed message to 'change_currency' topic.


    '''
    raise NotImplementedError()
"""

fixture_description = """
Invalid description
"""

relevant_examples = '''no examples passed'''

test_response = """
### application.py ###

print("invalid app code")
    
### test.py ###

import pytest

from faststream.kafka import TestKafkaBroker

from .app import CourseUpdates, broker, on_course_update


print("invalid test code")

"""


with TemporaryDirectory() as d:
    app_skeleton_file_name = Path(d) / APPLICATION_FILE_PATH
    write_file_contents(app_skeleton_file_name, fixture_skeleton_code)
    
    with mock_openai_create(test_response):    
        usage, is_valid_app_code = generate_app_and_test(fixture_description, OpenAIModel.gpt3.value, d, [], relevant_examples)
    
    logs_dir = Path(d) / LOGS_DIR_NAME
    assert logs_dir.exists()
    

assert int(usage[0]["total_tokens"]) > 0
print(usage)

print(is_valid_app_code)
assert not is_valid_app_code
assert isinstance(is_valid_app_code, bool)

⠹ Generating application and tests (usually takes around 30 to 90 seconds)... 

/home/harish/.local/lib/python3.11/site-packages/yaspin/core.py:119: UserWarning: color, on_color and attrs are not supported when running in jupyter
  self._color = self._set_color(color) if color else color


 ✘ Error: Failed to generate a valid application and test code.               
[defaultdict(<class 'int'>, {'prompt_tokens': 387, 'completion_tokens': 3, 'total_tokens': 390}), defaultdict(<class 'int'>, {'prompt_tokens': 387, 'completion_tokens': 3, 'total_tokens': 390}), defaultdict(<class 'int'>, {'prompt_tokens': 387, 'completion_tokens': 3, 'total_tokens': 390})]
False


/home/harish/.local/lib/python3.11/site-packages/yaspin/core.py:228: UserWarning: color, on_color and attrs are not supported when running in jupyter
  self._color = self._set_color(value) if value else value


In [ ]:
# | notest

# todo: fix the below test

fixture_skeleton_code = """
from pydantic import BaseModel, Field

from faststream import FastStream, Logger
from faststream.kafka import KafkaBroker


class Product(BaseModel):
    product_name: str = Field(..., description="Name of the product")
    currency: str = Field(..., description="Currency of the price")
    price: float = Field(..., description="Price of the product")


broker = KafkaBroker("localhost:9092")
app = FastStream(broker)


@broker.publisher("change_currency")
@broker.subscriber("store_product")
async def on_store_product(product: Product, logger: Logger) -> Product:
    '''Processes a message from 'store_product' topic, changes currency to 'EUR' and divides price by 7.5 if currency is 'HRK'.

    Instructions:
    1. Consume a message from 'store_product' topic.
    2. Log the consumed message using logger.info.
    3. Check if the currency attribute is set to 'HRK'.
    4. If the currency is 'HRK', change the currency to 'EUR' and divide the price by 7.5.
    5. If the currency is not 'HRK', do not modify the original message.
    6. Publish the consumed message to 'change_currency' topic.


    '''
    raise NotImplementedError()
"""

fixture_description = """
Create a FastStream application using localhost broker for testing and use default port number. It should consume from 'store_product' topic an JSON encoded object with the following three attributes: product_name, currency and price. The format of the currency will be three letter string, e.g. 'EUR'. For each consumed message, check if the currency attribute is set to 'HRK'. If it is then change the currency to 'EUR' and divide the price by 7.5, if the currency is not set to 'HRK' don't change the original message. Finally, publish the consumed message to 'change_currency' topic.
"""

test_response = '''

### application.py ###

from typing import Optional

from pydantic import BaseModel, Field

from faststream import FastStream, Logger
from faststream.kafka import KafkaBroker


class CourseUpdates(BaseModel):
    course_name: str = Field(..., examples=["Biology"], description="Course example")
    new_content: Optional[str] = Field(
        default=None, examples=["New content"], description="Content example"
    )


broker = KafkaBroker("localhost:9092")
app = FastStream(broker)


@broker.publisher("notify_updates")
@broker.subscriber("course_updates")
async def on_course_update(msg: CourseUpdates, logger: Logger) -> CourseUpdates:
    logger.info(msg)

    if msg.new_content:
        logger.info(f"Course has new content {msg.new_content=}")
        msg = CourseUpdates(
            course_name=("Updated: " + msg.course_name), new_content=msg.new_content
        )
    return msg
    
### test.py ###

import pytest

from faststream.kafka import TestKafkaBroker

from app.application import CourseUpdates, broker, on_course_update


@broker.subscriber("notify_updates")
async def on_notify_update(msg: CourseUpdates):
    pass


@pytest.mark.asyncio
async def test_app_without_new_content():
    async with TestKafkaBroker(broker):
        await broker.publish(CourseUpdates(course_name="Biology"), "course_updates")
        on_course_update.mock.assert_called_with(
            dict(CourseUpdates(course_name="Biology"))
        )
        on_notify_update.mock.assert_called_with(
            dict(CourseUpdates(course_name="Biology"))
        )


@pytest.mark.asyncio
async def test_app_with_new_content():
    async with TestKafkaBroker(broker):
        await broker.publish(
            CourseUpdates(
                course_name="Biology", new_content="We have additional classes..."
            ),
            "course_updates",
        )
        on_course_update.mock.assert_called_with(
            dict(
                CourseUpdates(
                    course_name="Biology", new_content="We have additional classes..."
                )
            )
        )
        on_notify_update.mock.assert_called_with(
            dict(
                CourseUpdates(
                    course_name="Updated: Biology",
                    new_content="We have additional classes...",
                )
            )
        )

'''

with TemporaryDirectory() as d:
    app_skeleton_file_name = Path(d) / APPLICATION_FILE_PATH
    write_file_contents(app_skeleton_file_name, fixture_skeleton_code)
    
    test_file_path = Path(d) / TEST_FILE_PATH
    test_file_path.parent.mkdir(parents=True, exist_ok=True)
    init_file_path = test_file_path.parent / "__init__.py"
    init_file_path.touch()
    
    with mock_openai_create(test_response):    
        usage, is_valid_app_code = generate_app_and_test(fixture_description, OpenAIModel.gpt3.value, d, [], relevant_examples)
    
    logs_dir = Path(d) / LOGS_DIR_NAME
    assert logs_dir.exists()
    

assert int(usage[0]["total_tokens"]) > 0
print(usage)

print(is_valid_app_code)
assert is_valid_app_code
assert isinstance(is_valid_app_code, bool)

⠹ Generating application and tests (usually takes around 30 to 90 seconds)... 

/home/harish/.local/lib/python3.11/site-packages/yaspin/core.py:119: UserWarning: color, on_color and attrs are not supported when running in jupyter
  self._color = self._set_color(color) if color else color


 ✘ Error: Failed to generate a valid application and test code.               
[defaultdict(<class 'int'>, {'prompt_tokens': 387, 'completion_tokens': 3, 'total_tokens': 390}), defaultdict(<class 'int'>, {'prompt_tokens': 387, 'completion_tokens': 3, 'total_tokens': 390}), defaultdict(<class 'int'>, {'prompt_tokens': 387, 'completion_tokens': 3, 'total_tokens': 390})]
False


/home/harish/.local/lib/python3.11/site-packages/yaspin/core.py:228: UserWarning: color, on_color and attrs are not supported when running in jupyter
  self._color = self._set_color(value) if value else value


AssertionError: 